In [1]:
import pandas as pd
import re
import hashlib

Formatar uma saída do log em json contendo a lista de request apresentada no log, cada objeto dentro da lista deve conter as
propriedades de uma entrada no log como remote_host, date, request, status_code, response_time, reffer, user_agent.

In [8]:
log_file_path = "test-access-001-1.log"
regex = '((^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}).*(\[([1-9].*?)\]).*"(GET.*?)".([1-9]\d\d).([0-9]*)."(ht.*?)")."(.*)"'

match_list = []
with open(log_file_path, "r") as file:
    for line in file:
        for match in re.finditer(regex, line, re.S):
            d = dict()
            d['remote_host'] = match.group(2)
            d['date'] = match.group(4)
            d['request'] =  match.group(5)
            d['status_code'] = match.group(6)
            d['response_time'] = match.group(7)
            d['reffer'] = match.group(8)
            d['user_agent'] = match.group(9)
            match_list.append(d)
            
match_list[:10]

#OBSERVAÇÃO: LIMITEI APENAS OS 10 PRIMEIROS PARA A VISUALIZAÇÃO FICAR COMPACTA, 
# PARA VISUALIZAR TODOS OS ELEMENTOS BASTA RETIRAR O [:10]

[{'remote_host': '192.141.19.154',
  'date': '12/Jan/2020 02:57:26 +0200',
  'request': 'GET /apache_pb.gif HTTP/1.1',
  'status_code': '200',
  'response_time': '79262',
  'reffer': 'http://localhost/manual/',
  'user_agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991'},
 {'remote_host': '192.220.104.143',
  'date': '12/Jan/2020 02:57:27 +0200',
  'request': 'GET /svn/filedetails.php?repname=Devel&path=/rpg/devel/game.pl&rev=0&sc=1 HTTP/1.1',
  'status_code': '200',
  'response_time': '52105',
  'reffer': 'http://localhost/~jan/WebSVN/listing.php?repname=Private&path=/&',
  'user_agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1 Mobile/15E148 Safari/604.1'},
 {'remote_host': '192.231.74.58',
  'date': '12/Jan/2020 02:57:28 +0200',
  'request': 'GET /manual/images/left.gif HTTP/1.1',
  'status_code': '200',
  'response_time': '83321',

Encontrar os 10 maiores tempos de resposta com sucesso do servidor na chamada GET /maunal/ com a origem do tráfego (reffer) igual a "http://localhost/svnview?repos=devel&rev=latest&root=SVNview/tmpl&list_revs=1"

In [3]:
logDataframe = pd.DataFrame(match_list)
logDataframe['unix_timestamp'] = pd.to_datetime(logDataframe.date)

httpFilter = 'http://localhost/svnview?repos=devel&rev=latest&root=SVNview/tmpl&list_revs=1'
requestFilter = '.*GET /manual/.*'

fasterResponseTime = logDataframe[
    (logDataframe.reffer == httpFilter) & 
    (logDataframe.request.str.contains(requestFilter))
]

fasterResponseTime = fasterResponseTime.sort_values(
    by=['response_time'], 
    ascending=False
)

fasterResponseTime.head(10)

,remote_host,date,request,status_code,response_time,reffer,user_agent,unix_timestamp
7723,192.68.214.83,11/Feb/2020 04:28:01 +0200,GET /manual/style/css/manual-loose-100pc.css H...,200,99764,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,2020-02-11 04:28:01+02:00
28384,192.17.108.230,26/Apr/2020 15:38:37 +0200,GET /manual/ HTTP/1.1,200,99646,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2020-04-26 15:38:37+02:00
14105,192.146.231.205,28/Feb/2020 11:52:42 +0200,GET /manual/style/css/manual.css HTTP/1.1,200,99446,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.6...,2020-02-28 11:52:42+02:00
14404,192.32.136.20,29/Feb/2020 06:16:49 +0200,GET /manual/images/favicon.ico HTTP/1.1,200,98821,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_6_1 like...,2020-02-29 06:16:49+02:00
27399,192.66.46.195,23/Apr/2020 13:21:44 +0200,GET /manual/style/css/manual-loose-100pc.css H...,200,98785,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2020-04-23 13:21:44+02:00
41821,192.140.80.188,16/Aug/2020 03:44:40 +0200,GET /manual/style/css/manual-loose-100pc.css H...,200,9823,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2020-08-16 03:44:40+02:00
11304,192.187.126.204,20/Feb/2020 20:16:45 +0200,GET /manual/style/css/manual-loose-100pc.css H...,200,97592,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (iPhone; CPU iPhone OS 11_3_1 like...,2020-02-20 20:16:45+02:00
24534,192.67.39.72,15/Apr/2020 15:08:59 +0200,GET /manual/images/feather.gif HTTP/1.1,200,97384,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2020-04-15 15:08:59+02:00
29626,192.250.181.236,29/Apr/2020 20:11:26 +0200,GET /manual/ HTTP/1.1,200,97036,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.6...,2020-04-29 20:11:26+02:00
36190,192.71.24.9,27/May/2020 00:51:04 +0200,GET /manual/ HTTP/1.0,200,96787,http://localhost/svnview?repos=devel&rev=lates...,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2020-05-27 00:51:04+02:00


Formatar uma saída em arquivo físico do access.log com a data em formato UNIX timestamp %Y-%m-%d %H:%M:%S e o IP convertido em um hash MD5

In [4]:
dataframeIpDate = logDataframe.copy()

dataframeIpDate['md5Hash']  = dataframeIpDate.remote_host.apply(lambda x: hashlib.md5(x.encode()).hexdigest())

dataframeIpDate = dataframeIpDate[['unix_timestamp','md5Hash']]

dataframeIpDate.to_csv('csvIpDate', sep='\t', index=False , header= False)

Formatar uma saída em arquivo físico agrupando a soma total de requests por dia do ano

In [5]:
yearDateRequest = logDataframe.copy()

yearDateRequest['year'] =  pd.DatetimeIndex(yearDateRequest.unix_timestamp).year
yearDateRequest['day'] = pd.DatetimeIndex(yearDateRequest.unix_timestamp).day

result = yearDateRequest.groupby(['year','day']).size().reset_index(name='counts')
result.to_csv('yearDateRequest', sep='\t', index=False , header= False)

Formatar uma saída em arquivo físico com endereços de IP únicos, um IP por linha, contidos no log com a última data de request realizado pelo remote IP

In [6]:
distinctIP = logDataframe.copy()

distinctIP = distinctIP[['remote_host','unix_timestamp']]

distinctIP = distinctIP.groupby(['remote_host']).max()

result.to_csv('distinctIP', sep='\t', index=False , header= False)